# script that makes all the plots that could be useful

this is a reference for whoever wants to get the plots that will be shown in the paper

In [1]:
import pandas as pd
import numpy as np
from multiprocessing import Pool
from functools import partial
import glob
import os
import plotly.express as px
import math
import matplotlib.pyplot as plt
from matplotlib import colors
import pybedtools as pbed
from scipy import stats, special
from statsmodels.stats import multitest
import statsmodels.api as sm
import statsmodels.formula.api as smf
import plotly.io as pio
import seaborn as sns

plt.rcParams['svg.fonttype'] = 'none'
base_dir = "http://bartzabel.ls.manchester.ac.uk/orozcolab/SNP2Mechanism/"

## plot expression of genes across different conditions

In [3]:
RNA_normalized_counts = pd.read_csv(f"{base_dir}/rna/RNA_normalized_counts.csv", index_col = 0)
metadata_RNA = pd.read_csv(f"{base_dir}/metadata/cleaned_RNA_metadata.csv", index_col = 0)

In [4]:
RNA_normalized_counts_melted = pd.melt(RNA_normalized_counts, id_vars=["ensembl","ENSG","symbol","genename","entrez"], 
        value_vars=RNA_normalized_counts.columns.difference(["ensembl","ENSG","symbol","genename","entrez"]),
        var_name="sample",value_name="expression")
RNA_normalized_counts_melted = RNA_normalized_counts_melted.merge(metadata_RNA, on = "sample")

In [9]:
def plot_across_celltypes(gene):
    filtered_df = RNA_normalized_counts_melted[RNA_normalized_counts_melted["symbol"] == gene]
    filtered_df["cell_type"] = filtered_df["cell_type"].map({"CD4": "CD4", "CD8": "CD8", "CD4_SF": "CD4", "CD8_SF": "CD8"})
    # Create the strip plot
    fig = px.strip(filtered_df, x="cell_type", y="expression", color="condition",
                hover_name="sample", hover_data=["expression"],
                color_discrete_map={"Healthy": "blue", "Diseased": "red"},
                title=f'Expression Levels of {gene} Across Cell Types',
                labels={"cell_type": "Cell Type", "expression": "Expression Level"})

    # Customizing the marker appearance
    fig.update_traces(marker=dict(size=8, opacity=0.8, line=dict(width=1, color='DarkSlateGrey')))
    fig.update_layout(
        width=600,  # Set the width of the figure in pixels
        height=700  # Set the height of the figure in pixels
    )

    fig.update_layout(
        title={'text': f'Expression Levels of {gene} Across samples', 'y':0.97, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
        xaxis_title="Cell Type",
        yaxis_title="Loop Strength",
        legend_title="Condition",
        font=dict(family="Arial, sans-serif", size=16),
        legend=dict(orientation="h", yanchor="bottom", y=1.01, xanchor="right", x=1.2)
    )
    fig.write_image(f"figures/expression_RNA_{gene}.svg",width=500, height=600)
    return fig

plot_across_celltypes("CD8A")
plot_across_celltypes("CD4")


/tmp/ipykernel_26230/1413736422.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_26230/1413736422.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## plot activty of ATAC-peak across conditions

In [12]:
ATAC_normalised_counts = pd.read_csv(f"{base_dir}/atac/merged/ATAC_DESeq2_quantile_normalized_counts.csv", index_col = 0)
metadata_ATAC = pd.read_csv(f"{base_dir}/metadata/cleaned_ATAC_metadata.csv", index_col = 0)

ATAC_normalised_counts_melted = pd.melt(ATAC_normalised_counts, id_vars=["CHR","START","END"], 
        value_vars=ATAC_normalised_counts.columns.difference(["CHR","START","END"]),
        var_name="sample",value_name="peak_height")
ATAC_normalised_counts_melted = ATAC_normalised_counts_melted.merge(metadata_ATAC, left_on = "sample", right_on = "id")

annotated_peaks = pd.read_csv(f"{base_dir}/atac/merged/annotated_consensus_peaks.csv", index_col = 0)
annotated_peaks["start"] = annotated_peaks["start"] - 1 

In [13]:
ATAC_normalised_counts_melted

,CHR,START,END,sample_x,peak_height,proper_name,id,patient,cell_type,condition,...,RF_positive,axial,peripheral,axial_peripheral,On_MTX,on_csDMARD,on_bDMARD_tsDMARD,on_steroid,previous_bDMARD_tsDMARD,group
0,chr1,9942,10442,NRHV014XCD4,25.840135,NRHV014XCD4,NRHV014XCD4,NRHV014X,CD4,healthy,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,-10
1,chr1,181066,181566,NRHV014XCD4,31.508040,NRHV014XCD4,NRHV014XCD4,NRHV014X,CD4,healthy,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,-10
2,chr1,191452,191952,NRHV014XCD4,30.571787,NRHV014XCD4,NRHV014XCD4,NRHV014X,CD4,healthy,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,-10
3,chr1,778488,778988,NRHV014XCD4,1112.893539,NRHV014XCD4,NRHV014XCD4,NRHV014X,CD4,healthy,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,-10
4,chr1,817112,817612,NRHV014XCD4,22.980808,NRHV014XCD4,NRHV014XCD4,NRHV014X,CD4,healthy,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13276993,chr9_KI270717v1_random,24847,25347,psa4951_CD8,9.272195,PSA4951CD8,psa4951_CD8,PSA4951,CD8,patient,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,10
13276994,chr9_KI270718v1_random,37612,38112,psa4951_CD8,20.429856,PSA4951CD8,psa4951_CD8,PSA4951,CD8,patient,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,10
13276995,chr9_KI270719v1_random,162917,163417,psa4951_CD8,14.291872,PSA4951CD8,psa4951_CD8,PSA4951,CD8,patient,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,10
13276996,chr9_KI270719v1_random,169793,170293,psa4951_CD8,45.143180,PSA4951CD8,psa4951_CD8,PSA4951,CD8,patient,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,10


In [18]:
def plot_peak_across_celltypes(gene, chrom, start):
    filtered_df = ATAC_normalised_counts_melted[(ATAC_normalised_counts_melted["CHR"] == chrom) & (ATAC_normalised_counts_melted["START"] == start)]
    filtered_df["cell_type"] = filtered_df["cell_type"].map({"CD4": "CD4", "CD8": "CD8", "CD4_SF": "CD4", "CD8_SF": "CD8"})
    # Create the strip plot
    fig = px.strip(filtered_df, x="cell_type", y="peak_height", color="condition",
                hover_name="proper_name", hover_data=["peak_height"],
                color_discrete_map={"Healthy": "blue", "Diseased": "red"},
                title=f'Chromatin accessibility Levels of {gene} promoter Across Cell Types',
                labels={"cell_type": "Cell Type", "peak_height": "Peak height"})

    # Customizing the marker appearance
    fig.update_traces(marker=dict(size=8, opacity=0.8, line=dict(width=1, color='DarkSlateGrey')))
    fig.update_layout(
        width=600,  # Set the width of the figure in pixels
        height=700  # Set the height of the figure in pixels
    )

    fig.update_layout(
        title={'text': f'Chromatin accessibility Levels of {gene} promoter Across Cell Types', 'y':0.97, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
        xaxis_title="Cell Type",
        yaxis_title="Peak Accessibility",
        legend_title="Condition",
        font=dict(family="Arial, sans-serif", size=16),
        legend=dict(orientation="h", yanchor="bottom", y=1.01, xanchor="right", x=1.2)
    )
    fig.write_image(f"figures/expression_ATAC_{gene}.svg",width=500, height=600)
    return fig

# plot of promoter of CD4 gene
plot_peak_across_celltypes("CD4","chr12", 6789245)
# plot of promoter of CD8 gene
plot_peak_across_celltypes("CD8A","chr2", 86808258)

/tmp/ipykernel_26230/3625484250.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_26230/3625484250.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## plot intensity of chromatin interactions across conditions

In [21]:
loops_analysed = pd.read_pickle(f"{base_dir}/hic/loops/aggregated_counts/aggregated_normalized_loops_CD4_CD8.pk")
metadata_hic = pd.read_csv(f"{base_dir}/metadata/cleaned_HiC_metadata.csv", index_col = 0)

column_name_dict = dict(zip(metadata_hic['folder_name'], metadata_hic['proper_name']))
loops_analysed = loops_analysed.rename(columns=column_name_dict)
loops_counts_melted = pd.melt(loops_analysed, id_vars=['chrA', 'A_start', 'A_end', 'chrB', 'B_start', 'B_end', 'FDR', 'DETECTION_SCALE', 'distance_bin'], 
        value_vars=loops_analysed.columns.difference(['chrA', 'A_start', 'A_end', 'chrB', 'B_start', 'B_end', 'FDR', 'DETECTION_SCALE', 'distance_bin']),
        var_name="proper_name",value_name="interaction_strength")
loops_counts_melted = loops_counts_melted.merge(metadata_hic[["patient","cell_type","condition","proper_name"]], on = "proper_name")


insulation_score_norm = pd.read_csv(f"{base_dir}/hic/insulation/aggregated_norm_ins_scores.csv.gz", index_col = 0)

insulation_score_norm = insulation_score_norm.rename(columns=column_name_dict)
insulation_score_melted = pd.melt(insulation_score_norm, id_vars=['chrom', 'start', 'end'], 
        value_vars=insulation_score_norm.columns.difference(['chrom', 'start', 'end']),
        var_name="proper_name",value_name="interaction_strength")
insulation_score_melted = insulation_score_melted.merge(metadata_hic[["patient","cell_type","condition","proper_name"]], on = "proper_name")

In [23]:
def plot_loop_celltypes(gene, chrom, start_A, start_B):
    filtered_df = loops_counts_melted[(loops_counts_melted["chrA"] == chrom) & (loops_counts_melted["A_start"] == start_A) & (loops_counts_melted["B_start"] == start_B)]
    filtered_df["cell_type"] = filtered_df["cell_type"].map({"CD4": "CD4", "CD8": "CD8", "CD4_SF": "CD4", "CD8_SF": "CD8"})
    # Create the strip plot
    fig = px.strip(filtered_df, x="cell_type", y="interaction_strength", color="condition",
                hover_name="proper_name", hover_data=["interaction_strength"],
                color_discrete_map={"Healthy": "blue", "Diseased": "red"},
                title=f'Loop Strength of {gene} loop across samples',
                labels={"cell_type": "Cell Type", "interaction_strength": "interaction strength"})

    # Customizing the marker appearance
    fig.update_traces(marker=dict(size=8, opacity=0.8, line=dict(width=1, color='DarkSlateGrey')))
    fig.update_layout(
        width=600,  # Set the width of the figure in pixels
        height=700  # Set the height of the figure in pixels
    )
    # Enhancing layout
    fig.update_layout(
        title={'text': f'Loop Strength of {gene} loop across samples', 'y':0.97, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
        xaxis_title="Cell Type",
        yaxis_title="Loop Strength",
        legend_title="Condition",
        font=dict(family="Arial, sans-serif", size=16),
        legend=dict(orientation="h", yanchor="bottom", y=1.01, xanchor="right", x=1.2)
    )

    # Show the plot
    fig.show()
    fig.write_image(f"figures/loop_strength_{gene}.svg",width=500, height=600)
    return fig

plot_loop_celltypes("CD4", "12", 6755000, 6825000)
plot_loop_celltypes("CD8A", "2", 86765000, 86805000)


/tmp/ipykernel_26230/2115016364.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



/tmp/ipykernel_26230/2115016364.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
def plot_ins_celltypes(chrom, start):
    fig = px.strip(insulation_score_melted[(insulation_score_melted["chrom"] == chrom) & (insulation_score_melted["start"] == start)], 
                   x="cell_type", y="interaction_strength", hover_name = "proper_name", title = f"insulation score of {chrom}:{start}", template = "plotly_white")
    return fig

plot_ins_celltypes("chr12", 6750000)
plot_ins_celltypes("chr2", 86750000)

# plotting by genotype. this will only work if you have the VCF file accessible

In [10]:
import vcf
def extract_id(string):
    return os.path.split(string)[1].split("_")[0]

variants = vcf.Reader(filename=f"/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_combined_analysis/QTL_tools/genotypes_HiC_ATAC_merged_0.05.vcf.gz")

In [11]:
#isolate record, gives a little bit of margin but still need to match rsid
def retrieve_genotypes(rs_number, chrom, rs_coord):
    sub_variants = variants.fetch(chrom, rs_coord - 10, rs_coord + 10)
    for tentative_var in sub_variants:
        if tentative_var.ID == rs_number:
            identified_var = tentative_var
            break
    pats = {}
    if rs_number != identified_var.ID:
        print("rs number not present in filtered annotated VCF file")
        raise Exception
    for s in identified_var.samples:
        pats[extract_id(s.sample)] = s.gt_type

    patients_genotypes = pd.DataFrame(pats.items(), columns=['patient', 'genotype'])
    return patients_genotypes


## gene expression (aka eQTL)

In [12]:
def visualize_eQTL(gene, rs_number, chrom, rs_coord, cell_types = ["CD4", "CD8"]):
    patients_genotypes = retrieve_genotypes(rs_number, chrom, rs_coord)
    counts_gene = RNA_normalized_counts_melted[(RNA_normalized_counts_melted["symbol"] == gene) & RNA_normalized_counts_melted["cell_type"].isin(cell_types)].merge(patients_genotypes, on='patient', how='left')
    fig = px.box(counts_gene, x="genotype", y="expression", color = "cell_type", hover_name = "proper_name", 
                   width=600, height=400, title = f"expression of {gene} by genotype of {rs_number}", template = "plotly_white")

    fig.update_traces(marker=dict(size=6, opacity=0.7), selector=dict(mode='markers'))
    return fig

In [ ]:
visualize_eQTL("FADS1","rs968567", "chr11", 61828092).write_image(f"{base_dir}/integration_analysis/figures/eQTL_rs968567_FADS1.svg")
visualize_eQTL("FADS2","rs968567", "chr11", 61828092).write_image(f"{base_dir}/integration_analysis/figures/eQTL_rs968567_FADS2.svg")
visualize_eQTL("FADS3","rs968567", "chr11", 61828092)

## chromatin accessibility (aka caQTL)

In [13]:
def visualize_caQTL(chrom, start, rs_number, rs_coord, cell_types = ["CD4", "CD8"]):
    patients_genotypes = retrieve_genotypes(rs_number, chrom, rs_coord)
    counts_peak = ATAC_normalised_counts_melted[(ATAC_normalised_counts_melted["CHR"] == chrom) & (ATAC_normalised_counts_melted["START"] == start) & ATAC_normalised_counts_melted["cell_type"].isin(cell_types)].merge(patients_genotypes, on='patient', how='left')
    fig = px.box(counts_peak, x="genotype", y="peak_height", color = "cell_type", hover_name = "proper_name", 
                   width=600, height=400, title = f"peak height of {chrom}:{start} by genotype of {rs_number}", template = "plotly_white")

    fig.update_traces(marker=dict(size=6, opacity=0.7), selector=dict(mode='markers'))
    return fig
visualize_caQTL("chr5", 132369456,"rs2631367", 132369766)

In [14]:
fig = visualize_caQTL("chr11", 95578036,"rs4409785", 	95578258)
fig.write_image(f"{base_dir}/integration_analysis/figures/caQTL_rs4409785.svg")

In [15]:
fig = visualize_caQTL("chr17", 39872642,"rs12936231", 39872866)
fig.write_image(f"{base_dir}/integration_analysis/figures/caQTL_rs12936231.svg")

In [16]:
fig = visualize_caQTL("chr19", 39440211,"rs2353678", 39440419)
fig.write_image(f"{base_dir}/integration_analysis/figures/caQTL_rs2353678.svg")

In [17]:
fig = visualize_caQTL("chr11", 61816418,"rs968567", 61828092)
fig.write_image(f"{base_dir}/integration_analysis/figures/caQTL_1_rs968567.svg")
fig = visualize_caQTL("chr11", 61827746,"rs968567", 61828092)
fig.write_image(f"{base_dir}/integration_analysis/figures/caQTL_2_rs968567.svg")
fig = visualize_caQTL("chr11", 61834520,"rs968567", 61828092)
fig.write_image(f"{base_dir}/integration_analysis/figures/caQTL_3_rs968567.svg")
fig = visualize_caQTL("chr11", 61871089,"rs968567", 61828092)
fig.write_image(f"{base_dir}/integration_analysis/figures/caQTL_4_rs968567.svg")

## looping (aka loopQTL)

In [14]:
def visualize_loopQTL(chr_A, start_A, start_B, rs_number, chrom, rs_coord, cell_types = ["CD4", "CD8"]):
    patients_genotypes = retrieve_genotypes(rs_number, chrom, rs_coord)
    counts_peak = loops_counts_melted[(loops_counts_melted["chrA"] == chr_A) & (loops_counts_melted["A_start"] == start_A) & (loops_counts_melted["B_start"] == start_B) & loops_counts_melted["cell_type"].isin(cell_types)].merge(patients_genotypes, on='patient', how='left')
    fig = px.box(counts_peak, x="genotype", y="interaction_strength", color = "cell_type", hover_name = "proper_name", 
                   width=600, height=400, title = f"loop strength of {chrom}:{start_A}-{start_B} by genotype of {rs_number}", template = "plotly_white")

    fig.update_traces(marker=dict(size=6, opacity=0.7), selector=dict(mode='markers'))
    return fig
visualize_loopQTL("17",39872500,39977500,"rs12936231", "chr17", 39872866)

In [19]:
fig = visualize_loopQTL("17",39872500,39977500,"rs12936231", "chr17", 39872866)
fig.write_image(f"{base_dir}/integration_analysis/figures/loopQTL_rs12936231.svg")

## insulation score (aka insQTL)

In [15]:
def visualize_insQTL(start, rs_number, chrom, rs_coord, cell_types = ["CD4", "CD8"]):
    patients_genotypes = retrieve_genotypes(rs_number, chrom, rs_coord)
    counts_ins = insulation_score_melted[(insulation_score_melted["chrom"] == chrom) & (insulation_score_melted["start"] == start) & insulation_score_melted["cell_type"].isin(cell_types)].merge(patients_genotypes, on='patient', how='left')
    fig = px.strip(counts_ins, x="genotype", y="interaction_strength", color = "cell_type", hover_name = "proper_name", 
                   width=800, height=400, title = f"insulation score of {chrom}:{start} by genotype of {rs_number}", template = "plotly_white")

    fig.update_traces(marker=dict(size=6, opacity=0.7), selector=dict(mode='markers'))
    return fig
visualize_insQTL(132350000,"rs2631367", "chr5", 132369766)

# plotting between two modalities

## gene vs loop

In [16]:
def plot_gene_vs_loop(gene, chr_A, start_A, start_B, cell_types = ["CD4", "CD8"]):
    loop_intensities = loops_counts_melted[(loops_counts_melted["chrA"] == chr_A) & (loops_counts_melted["A_start"] == start_A) & (loops_counts_melted["B_start"] == start_B) & loops_counts_melted["cell_type"].isin(cell_types)]
    RNA_counts = RNA_normalized_counts_melted[(RNA_normalized_counts_melted["symbol"] == gene) & RNA_normalized_counts_melted["cell_type"].isin(cell_types)]
    merged_df = RNA_counts.merge(loop_intensities, on = ["patient","cell_type","proper_name"])

    fig = px.scatter(merged_df, x="expression", y="interaction_strength", color = "cell_type", hover_name = "proper_name", 
                    width=800, height=400, 
                    title = f"loop strength {chr_A}:{start_A}-{start_B} by expression of {gene}", 
                    trendline = "ols", trendline_scope="overall", template = "plotly_white")
    return fig
plot_gene_vs_loop("TNIP1", "5", 151050000, 151115000)

## ATAC-peak vs loop

In [48]:
def plot_ATAC_vs_loop(chrom, start, chr_A, start_A, start_B, cell_types = ["CD4", "CD8"]):
    loop_intensities = loops_counts_melted[(loops_counts_melted["chrA"] == chr_A) & (loops_counts_melted["A_start"] == start_A) & (loops_counts_melted["B_start"] == start_B) & loops_counts_melted["cell_type"].isin(cell_types)]
    counts_peak = ATAC_normalised_counts_melted[(ATAC_normalised_counts_melted["CHR"] == chrom) & (ATAC_normalised_counts_melted["START"] == start) & ATAC_normalised_counts_melted["cell_type"].isin(cell_types)]
    merged_df = counts_peak.merge(loop_intensities, on = ["patient","cell_type","proper_name"])

    fig = px.scatter(merged_df, x="peak_height", y="interaction_strength", hover_name = "proper_name", 
                    width=800, height=400, 
                    title = f"loop strength {chr_A}:{start_A}-{start_B} vs peak height of {chrom}:{start}", 
                    trendline = "ols",trendline_scope="overall", template = "plotly_white")
    return fig
fig = plot_ATAC_vs_loop("chr2", 110858286, "2", 110852500, 111120000)
fig.write_image(f"{base_dir}/integration_analysis/figures/BCL2L11_loop_vs_enhancer.svg")

## gene vs ATAC

In [17]:
def plot_gene_vs_ATAC(chrom, start, gene, cell_types = ["CD4", "CD8"]):
    RNA_counts = RNA_normalized_counts_melted[(RNA_normalized_counts_melted["symbol"] == gene) & RNA_normalized_counts_melted["cell_type"].isin(cell_types)]
    counts_peak = ATAC_normalised_counts_melted[(ATAC_normalised_counts_melted["CHR"] == chrom) & (ATAC_normalised_counts_melted["START"] == start) & ATAC_normalised_counts_melted["cell_type"].isin(cell_types)]
    merged_df = counts_peak.merge(RNA_counts, on = ["patient","cell_type","proper_name"])

    fig = px.scatter(merged_df, x="peak_height", y="expression", color = "cell_type", hover_name = "proper_name", 
                    width=800, height=400, 
                    title = f"peak height of {chrom}:{start} vs expression of {gene}", 
                    trendline = "ols",trendline_scope="overall", template = "plotly_white")
    return fig
plot_gene_vs_ATAC("chr2", 110858286, "BUB1")

## plot two ATAC peaks

In [20]:
def plot_ATAC_vs_ATAC(chrom, start, chrom_B, start_B, cell_types = ["CD4", "CD8"]):
    counts_peak = ATAC_normalised_counts_melted[(ATAC_normalised_counts_melted["CHR"] == chrom) & (ATAC_normalised_counts_melted["START"] == start) & ATAC_normalised_counts_melted["cell_type"].isin(cell_types)]
    counts_peak_B = ATAC_normalised_counts_melted[(ATAC_normalised_counts_melted["CHR"] == chrom_B) & (ATAC_normalised_counts_melted["START"] == start_B) & ATAC_normalised_counts_melted["cell_type"].isin(cell_types)]
    merged_df = counts_peak.merge(counts_peak_B, on = ["patient","cell_type","proper_name"])

    fig = px.scatter(merged_df, x="peak_height_x", y="peak_height_y", color = "cell_type", hover_name = "proper_name", 
                    width=800, height=400, 
                    title = f"peak height of {chrom}:{start} vs peak height of {chrom_B}:{start_B}", 
                    trendline = "ols",trendline_scope="overall", template = "plotly_white")
    return fig
plot_ATAC_vs_ATAC("chr2", 110858286, "chr2", 111116949)

# plot specific things for specific regions

In [21]:
plot_ATAC_vs_loop("chr4", 122578373, "4", 122580000, 122717500, cell_types = ["CD4", "CD8", "CD8_SF", "CD4_SF"])

In [22]:
visualize_caQTL("chr4", 122578373, "rs13140464", 122578589)

In [50]:
fig = visualize_caQTL("chr2", 110858286, "rs13401811", 110858526)
fig.write_image(f"{base_dir}/integration_analysis/figures/BCL2L11_SNP_vs_enhancer.svg")

## code that allows to extract specific ATAC regions (not called peaks)

In [28]:
import sys
sys.path.append(f"{base_dir}/data_functions")
from extract_custom_ATAC_pos import get_region
def visualize_caQTL_custom(chrom, start, end, rs_number, rs_coord, cell_types = ["CD4", "CD8"]):
    patients_genotypes = retrieve_genotypes(rs_number, chrom, rs_coord)
    counts = get_region(chrom, start, end)
    counts = counts.merge(metadata_ATAC, on = "proper_name")
    counts_peak = counts[counts["cell_type"].isin(cell_types)].merge(patients_genotypes, on='patient', how='left')
    fig = px.box(counts_peak, x="genotype", y="peak_height", color = "cell_type", hover_name = "proper_name", points = "all",
                   width=800, height=400, title = f"peak height of {chrom}:{start} by genotype of {rs_number}", template = "plotly_white")

    fig.update_traces(marker=dict(size=6, opacity=0.7), selector=dict(mode='markers'))
    return fig

visualize_caQTL_custom("chr4", 122577957, 122578085,"rs13140464", 122578589)

INFO: Pandarallel will run on 28 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [29]:
def visualize_custom_two_ca(chrom, start, end,chrom_B, start_B, end_B):
    counts_A = get_region(chrom, start, end).merge(metadata_ATAC, on = "proper_name")
    counts_B = get_region(chrom_B, start_B, end_B)
    df = counts_A.merge(counts_B, on = "proper_name")
    fig = px.strip(df, x="peak_height_x", y="peak_height_y", color = "cell_type", hover_name = "proper_name", 
                   width=800, height=400, title = f"peak height {chrom}:{start} vs {chrom_B}:{start_B}", template = "plotly_white")

    fig.update_traces(marker=dict(size=6, opacity=0.7), selector=dict(mode='markers'))
    return fig
visualize_custom_two_ca("chr4", 122577957, 122578085,"chr4", 122578307,122578807)

In [37]:
visualize_custom_two_ca("chr2", 110858286, 110858786,"chr2", 111122473,111122721)

In [25]:
visualize_caQTL_custom("chr17", 39926950,39927103, "rs12936231", 39872867)

In [26]:
visualize_caQTL("chr17",39927312, "rs12936231", 39872867)

chron's CTCF locus

In [27]:
visualize_caQTL_custom("chr5",150846998, 150847045, "rs17111376", 150847337) # SNP upstream of CTCF peak 1

In [28]:
visualize_caQTL_custom("chr5",150847057,150847099, "rs17111376", 150847337) # SNP upstream of CTCF peak 2

In [29]:
visualize_caQTL_custom("chr5",150847313,150847360, "rs17111376", 150847337) # SNP downstream of CTCF peak

In [29]:
visualize_caQTL_custom("chr5",150847175,150847225, "rs17111376", 150847337) # middle of CTCF peak

In [30]:
visualize_caQTL_custom("chr5",150846774,150846789, "rs17111376", 150847337) # SNP upstream of CTCF peak even more upstream

In [31]:
visualize_caQTL_custom("chr19",39440318,39440519, "rs2353678", 39440418) # SNP directly overlapping CTCF motif

In [32]:
visualize_caQTL_custom("chr11",95578058,95578458, "rs4409785", 95578258) # SNP directly overlapping CTCF motif RA locus 